In [115]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [116]:
xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\test.csv', delimiter=',', dtype=np.float32)
x_data=xy[:,0:-1]
y_data=xy[:,[-1]]
F_data=xy[:,:]

In [117]:
# Model Inputs
def model_inputs(real_dim, noise_dim):
    inputs_real_ = tf.placeholder(tf.float32, shape=[None, real_dim], name='inputs_real')
    inputs_z_ = tf.placeholder(tf.float32, shape=[None, noise_dim], name='inputs_z')
    
    return inputs_real_, inputs_z_

def leaky_relu(x, alpha):
    return tf.maximum(alpha * x, x)

In [118]:
# Generator Network# Genera 
def model_generator(z_input, out_dim, n_units=128, reuse=False, alpha=0.01):
    # used to reuse variables, name scope
    with tf.variable_scope('generator', reuse=reuse):
        hidden_layer = tf.layers.dense(z_input, n_units, activation=tf.nn.relu)
        
        logits = tf.layers.dense(hidden_layer, out_dim, activation=None)
        outputs = tf.nn.tanh(logits)
        
        return outputs, logits

In [119]:
# Discriminator Network
def model_discriminator(input, n_units=128, reuse=False, alpha=0.01):
    with tf.variable_scope('discriminator', reuse=reuse):
        hidden_layer = tf.layers.dense(input, n_units, activation=None)
        hidden_layer = leaky_relu(hidden_layer, alpha)
        
        logits = tf.layers.dense(hidden_layer, 1, activation=None)
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [120]:
input_size = 42
z_dim = 21
g_hidden_size = 128
d_hidden_size = 128
alpha = 0.01
smooth = 0.1
learning_rate = 0.01

In [122]:
tf.reset_default_graph()  # If we don't have this, as we call this block over and over, the graph gets bigger and bigger

graph = tf.Graph()
with graph.as_default():
    inputs_real, inputs_z = model_inputs(input_size, z_dim)
    
    g_outputs, g_logits = model_generator(inputs_z, input_size, n_units=g_hidden_size, reuse=False, alpha=alpha)
    
    d_outputs_real, d_logits_real = model_discriminator(inputs_real, n_units=d_hidden_size, reuse=False, alpha=alpha)
    d_outputs_fake, d_logits_fake = model_discriminator(g_outputs, n_units=d_hidden_size, reuse=True, alpha=alpha)
    
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_logits_real) * (1-smooth)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_logits_fake)))
    
    d_loss = d_loss_real + d_loss_fake
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_logits_fake)))
    
    t_vars = tf.trainable_variables()
    g_vars = [variable for variable in t_vars if 'generator' in variable.name]
    d_vars = [variable for variable in t_vars if 'discriminator' in variable.name]
    
    # Affected Variables with var_list
    d_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(d_loss, var_list=d_vars)
    g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(g_loss, var_list=g_vars)
    
    # Saving variables with var_list
    saver = tf.train.Saver(var_list=g_vars)

array([[ 0.24156609,  0.00700332, -0.24801628, -0.4167633 ,  0.24545178,
         0.21338429,  0.0299005 , -0.16820495,  0.10001699, -0.08505337,
         0.14495878, -0.39721113,  0.02750633,  0.4690831 ,  0.00939092,
        -0.24894261, -0.12272524,  0.04453686, -0.25118318, -0.11378437,
         0.12798975, -0.165945  , -0.5535513 ,  0.03263574, -0.07883901,
         0.15392081, -0.20994031, -0.1039602 ,  0.31484488, -0.0818314 ,
        -0.3927617 , -0.08522824,  0.039054  , -0.01729485,  0.35585374,
        -0.00151189,  0.311337  ,  0.09941055,  0.07145674,  0.24866717,
        -0.17604522,  0.20155935]], dtype=float32)

In [125]:
samples = []
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(500):
        
        batch_images = F_data[step].reshape([1, 42])
        batch_z = np.random.uniform(-1, 1, size=[1, z_dim])
        
        _ = sess.run(d_optimizer, feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        _ = sess.run(g_optimizer, feed_dict={inputs_z : batch_z})
        loss_d, loss_g = sess.run([d_loss, g_loss], feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        if step%100==0:
            print('step {} / {} Complete. D_Loss : {:0.3f}, G_Loss : {:0.3f}'.format(step+1, 10000, loss_d, loss_g))
        
        sample_z = np.random.uniform(-1, 1, size=[1, z_dim])  # 16 Samples each epoch
        gen_samples, _ = sess.run(model_generator(inputs_z, input_size, reuse=True), feed_dict={inputs_z : sample_z})

        samples.append(gen_samples)

    print('Training Complete. ')

step 1 / 10000 Complete. D_Loss : 8.364, G_Loss : 0.554
step 101 / 10000 Complete. D_Loss : 11.305, G_Loss : 5.947
step 201 / 10000 Complete. D_Loss : 9.947, G_Loss : 3.006
step 301 / 10000 Complete. D_Loss : 13.842, G_Loss : 5.885
step 401 / 10000 Complete. D_Loss : 22.200, G_Loss : 4.023
Training Complete. 


In [165]:
#xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\test.csv', delimiter=',', dtype=np.float32)
#x_data=xy[:,0:-1]
#y_data=xy[:,[-1]]
#F_data=xy[:,:]

x1_data=gen_samples[:,0:-1]
y1_data=gen_samples[:,[-1]]



nb_classes=7

X=tf.placeholder(tf.float32,[None,41])
Y=tf.placeholder(tf.int32,[None,1])

Y_one_hot=tf.one_hot(Y,nb_classes)
Y_one_hot=tf.reshape(Y_one_hot,[-1,nb_classes])

W=tf.Variable(tf.random_normal([41,nb_classes]),name='weight')
b=tf.Variable(tf.random_normal([nb_classes]),name='bias')

logits=tf.matmul(X,W)+b
hypothesis=tf.nn.softmax(logits)

cost_i=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y_one_hot)

cost=tf.reduce_mean(cost_i)
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction=tf.argmax(hypothesis,1) #가능성을 퍼센트로~~
correct_prediction=tf.equal(prediction,tf.arg_max(Y_one_hot,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10000):
        sess.run(optimizer,feed_dict={X:x1_data,Y:y1_data})
        if step %100==0:
            loss,acc=sess.run([cost,accuracy],feed_dict={X:x_data,Y:y_data})
            print("step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step,loss,acc))
 
    print("training by gan sample")
  

    pred = sess.run(prediction, feed_dict={X: x_data})
    for p, y in zip(pred, y_data.flatten()):
            print("[{}] Prediction: {} Real Y: {}".format(p == int(y), p, int(y)))
            


step:     0	Loss: 59159.953	Acc: 0.00%
step:   100	Loss: 50710.336	Acc: 0.09%
step:   200	Loss: 50469.438	Acc: 0.10%
step:   300	Loss: 50329.664	Acc: 0.10%
step:   400	Loss: 50230.805	Acc: 0.11%
step:   500	Loss: 50154.309	Acc: 0.12%
step:   600	Loss: 50091.832	Acc: 0.12%
step:   700	Loss: 50039.090	Acc: 0.13%
step:   800	Loss: 49993.477	Acc: 0.13%
step:   900	Loss: 49953.199	Acc: 0.13%
step:  1000	Loss: 49917.195	Acc: 0.13%
step:  1100	Loss: 49884.684	Acc: 0.14%
step:  1200	Loss: 49854.969	Acc: 0.14%
step:  1300	Loss: 49827.637	Acc: 0.14%
step:  1400	Loss: 49802.402	Acc: 0.14%
step:  1500	Loss: 49778.910	Acc: 0.15%
step:  1600	Loss: 49756.930	Acc: 0.15%
step:  1700	Loss: 49736.262	Acc: 0.15%
step:  1800	Loss: 49716.828	Acc: 0.15%
step:  1900	Loss: 49698.414	Acc: 0.15%
step:  2000	Loss: 49680.980	Acc: 0.15%
step:  2100	Loss: 49664.355	Acc: 0.15%
step:  2200	Loss: 49648.543	Acc: 0.15%
step:  2300	Loss: 49633.402	Acc: 0.15%
step:  2400	Loss: 49618.984	Acc: 0.15%
step:  2500	Loss: 49605.0

[False] Prediction: 4 Real Y: 1
[False] Prediction: 5 Real Y: 0
[False] Prediction: 5 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] Prediction: 5 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 5 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 6 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 5 Real Y: 0
[False] 

[False] Prediction: 4 Real Y: 1
[False] Prediction: 5 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 5 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 5 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 5 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] 

[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 5 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 5 Real Y: 1
[False] Prediction: 5 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 5 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] 

[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 5 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] 

...

[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 5 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] 

[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 5 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[True] Prediction: 0 Real Y: 0
[False] P

[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 5 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 5 Real Y: 0
[False] 

[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 5 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 6 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] 

[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 5 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 1
[False] Prediction: 4 Real Y: 1
[False] Prediction: 5 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 6 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 0
[False] Prediction: 4 Real Y: 1
[False] Prediction: 4 Real Y: 0
[False] 

array([[-0.9994763]], dtype=float32)